In [1]:
load('utils.sage') # log2_strict

We are working over GF(17), so RS codes will only work for block sizes <=17.

In [2]:
F = GF(17)
R.<X> = F[]
EF.<z> = F.extension(X^2 - 3)
F, EF

(Finite Field of size 17, Finite Field in z of size 17^2)

In [3]:
def to_bits(x, n_bits):
    for i in range(n_bits):
        yield (x >> i) & 1
def eq(ys, R=None):
    ys = list(ys)
    if R is None:
        R = PolynomialRing(ys[0].parent(), len(ys), 'x')
    return product(
        x*y + (1 - x)*(1 - y)
        for x, y in zip(R.gens(), ys)
    )
def mle(evals):
    n_bits = log2_strict(len(evals))
    return sum(eq(to_bits(i, n_bits))*evals[i] for i in range(2^n_bits))

# Basic Small-Field Construction
> (..) we allow the polynomial’s coefficient field and the code’s alphabet to be sub-cryptographically sized, though we require that these fields be equal to each other.

In [4]:
def ExtensionCode(f, n, k):
    rs = codes.ReedSolomonCode(f, n, k)
    def encode(v):
        d = v.base_ring().degree()
        if d == 1:
            return rs.encode(v)
        else:
            codewords = [rs.encode(vector([x[i] for x in v])) for i in range(d)]
            return vector([v.base_ring()(list(xs)) for xs in zip(*codewords)])
    return encode

In [5]:
l0, l1 = (1, 3)
l = l0 + l1
m0, m1 = 2^l0, 2^l1
C = ExtensionCode(F, 2^l, m1)

In [6]:
with seed(0):
    evals = [F.random_element() for _ in range(2^l)]
evals

[3, 16, 1, 0, 10, 16, 5, 16, 2, 5, 14, 2, 8, 4, 4, 6]

In [7]:
[mle(evals)(*to_bits(i, 4)) for i in range(16)]

[3, 16, 1, 0, 10, 16, 5, 16, 2, 5, 14, 2, 8, 4, 4, 6]

### Commit to `evals`

In [8]:
t = matrix(m0, m1, evals); t

[ 3 16  1  0 10 16  5 16]
[ 2  5 14  2  8  4  4  6]

In [9]:
u = matrix([C(row) for row in t]); u

[16 11  7 14 11  3  3 10  5 15 11  5  3 10  2  7]
[11  5  5  4  5  5 13 13 11  1  5 12 13  5  4  5]

In [10]:
# todo: merkle commit u

### Open `evals` at `r` = `s`

In [11]:
with seed(1):
    rs = [EF.random_element() for _ in range(l)]
s = mle(evals)(*rs); s
rs, s

([2*z + 11, z + 11, 3*z, 8*z + 10], 11)

In [12]:
r0_expansion = vector([eq(rs[:l1])(*to_bits(i, l1)) for i in range(m1)])
r1_expansion = vector([eq(rs[l1:])(*to_bits(i, l0)) for i in range(m0)])
print(r0_expansion, r1_expansion)
t_prime = r1_expansion * t
t_prime

(z + 6, 11*z + 10, 10*z + 2, 9*z, 12*z + 15, 9*z + 10, 9*z + 1, 7*z + 8) (9*z + 8, 8*z + 10)


(9*z + 10, 14*z + 8, 2*z + 12, 16*z + 3, z + 7, 6*z + 15, 9*z + 12, 5*z + 1)

### Verify

In [13]:
assert t_prime * r0_expansion == s
for j in [randint(0, 2^l - 1) for _ in range(5)]:
    # todo: merkle check u.column(j)
    assert r1_expansion * u.column(j) == C(t_prime)[j]

# Block-Level Encoding
> (..) suitable for polynomials over fields smaller than the alphabet of the linear block code selected for use.

In [14]:
def PackedCode(ef, n, k):
    d = ef.degree()
    assert n % d == 0 and k % d == 0
    rs = codes.ReedSolomonCode(ef, n, k//d)
    print(rs)
    def encode(v):
        if v.base_ring() is ef:
            return rs.encode(v)
        else:
            assert v.base_ring() is ef.base_ring() and len(v) % d == 0
            packed = vector([ef(v[i:i+d]) for i in range(0, len(v), d)])
            return rs.encode(packed)
    return encode

In [15]:
# since our trace length is 2^(3+2) = 32, we would not be able to apply a code in GF(17)
l0, l1 = (1, 4)
l = l0 + l1
m0, m1, n = 2^l0, 2^l1, 2^l
C = PackedCode(EF, n, m1)
m0, m1, n

[32, 8, 25] Reed-Solomon Code over GF(289)


(2, 16, 32)

In [16]:
with seed(0):
    evals = [F.random_element() for _ in range(n)]
print(evals)

[3, 16, 1, 0, 10, 16, 5, 16, 2, 5, 14, 2, 8, 4, 4, 6, 0, 4, 15, 15, 10, 12, 13, 6, 13, 7, 1, 9, 1, 15, 12, 8]


### Commit

In [17]:
t = matrix(m0, m1, evals); print(t)
assert (t.nrows(), t.ncols()) == (m0, m1)

[ 3 16  1  0 10 16  5 16  2  5 14  2  8  4  4  6]
[ 0  4 15 15 10 12 13  6 13  7  1  9  1 15 12  8]


In [18]:
u = matrix([C(row) for row in t]); print(u)
assert (u.nrows(), u.ncols()) == (m0, n)
# todo: merkle commit u

[14*z + 13   6*z + 4  15*z + 4  9*z + 11 10*z + 10  12*z + 1 12*z + 15   4*z + 8 13*z + 14  8*z + 16  16*z + 7   2*z + 6        10   3*z + 9  16*z + 9 15*z + 11        16   8*z + 5 11*z + 15  13*z + 5    z + 10 11*z + 11         9 13*z + 12  6*z + 11     z + 8  10*z + 5         8  16*z + 8  11*z + 4 14*z + 11  4*z + 14]
[ 8*z + 14   4*z + 2  3*z + 13   3*z + 6  16*z + 7       5*z       2*z   9*z + 1  12*z + 4  11*z + 8  6*z + 15 16*z + 12   6*z + 4     z + 8   3*z + 8      14*z         0 13*z + 16   3*z + 8  14*z + 3  2*z + 14  2*z + 11 12*z + 14   4*z + 8       7*z       2*z 11*z + 11 12*z + 11  15*z + 8  11*z + 6  9*z + 16 11*z + 10]


### Open

In [19]:
with seed(1):
    rs = [EF.random_element() for _ in range(l)]
s = mle(evals)(*rs); s
rs, s

([2*z + 11, z + 11, 3*z, 8*z + 10, 16*z + 3], 2*z + 3)

>  P computes the matrix–vector product t' := ⊗_{i=l1..l-1} (1 − r_i, r_i) · t, here interpreting the
> matrix t as an unpacked, m0 × m1 matrix with entries in Tι. P sends V t' in the clear.

In [20]:
r0_expansion = vector([eq(rs[:l1])(*to_bits(i, l1)) for i in range(m1)])
r1_expansion = vector([eq(rs[l1:])(*to_bits(i, l0)) for i in range(m0)])
print(r0_expansion, r1_expansion)
t_prime = r1_expansion * t
t_prime

(11*z + 7, 8*z + 3, 13*z + 14, 4*z + 5, 10*z + 2, 9*z, 13*z + 13, 9*z + 15, 7*z + 16, 3*z + 7, 14*z + 5, 5*z + 12, 2*z + 13, 10, 13*z + 5, 15*z + 10) (z + 15, 16*z + 3)


(3*z + 11, 12*z + 14, 3*z + 9, 2*z + 11, 10, 4*z + 4, 9*z + 12, 10*z + 3, 6*z + 1, 15*z + 11, 13*z + 9, 10*z + 6, 7*z + 4, 6*z + 3, 9*z + 11, 15*z + 12)

### Verify

Now we reinterpret t_prime in the same way as `ExtensionCode`: as a vector of a higher extension, that is recombined with the extension generator

In [21]:
assert t_prime * r0_expansion == s
for j in [randint(0, 2^l - 1) for _ in range(10)]:
    # todo: merkle check u.column(j)
    u_prime_0 = vector([x for x in t_prime[::2]])
    u_prime_1 = vector([x for x in t_prime[1::2]])
    assert r1_expansion * u.column(j) == C(u_prime_0)[j] + z * C(u_prime_1)[j]